<a href="https://colab.research.google.com/github/KyokoM/tsunodalab/blob/master/deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#GPUの確認
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
import google.colab
import googleapiclient.discovery
import googleapiclient.http

In [0]:
googledriveに認証接続
google.colab.auth.authenticate_user()
drive_service = googleapiclient.discovery.build('drive', 'v3')

In [0]:
#colabにfile取り込み
upload_filename = 'GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct'

In [0]:
file_list = drive_service.files().list(q="name='" + upload_filename + "'").execute().get('files')

In [40]:
file_id = None
for file in file_list:
  if file.get('name') == upload_filename:
    file_id = file.get('id')
    break

if file_id is None:
  # ファイル ID を取得できなかった場合はエラーメッセージを出力します。
  print(upload_filename + ' が見つかりません.')
else:
  # colab 環境へファイルをアップロードします。
  with open(upload_filename, 'wb') as f:
    request = drive_service.files().get_media(fileId=file_id)
    media = googleapiclient.http.MediaIoBaseDownload(f, request)

    done = False
    while not done:
      progress_status, done = media.next_chunk()
      print(100*progress_status.progress(), end="")
      print("%完了")

  print('GoogleドライブからColab環境へのファイル取り込みが完了しました.')

3.724801447200384%完了
7.449602894400768%完了
11.174404341601154%完了
14.899205788801536%完了
18.624007236001923%完了
22.348808683202307%完了
26.07361013040269%完了
29.798411577603073%完了
33.52321302480346%完了
37.248014472003845%完了
40.97281591920423%完了
44.697617366404614%完了
48.42241881360499%完了
52.14722026080538%完了
55.87202170800577%完了
59.596823155206145%完了
63.32162460240654%完了
67.04642604960692%完了
70.7712274968073%完了
74.49602894400769%完了
78.22083039120807%完了
81.94563183840846%完了
85.67043328560884%完了
89.39523473280923%完了
93.1200361800096%完了
96.84483762720998%完了
100.0%完了
GoogleドライブからColab環境へのファイル取り込みが完了しました.


In [41]:
ls

data/
GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct
GTEx_v7_Annotations_SampleAttributesDS.txt
seq_data.zip


In [0]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
import pandas as pd
import h5py
import os
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.svm import LinearSVC

from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from IPython.display import Image, display_png
#import pydotplus as pdp

from sklearn.ensemble import RandomForestClassifier

import seaborn as sns

In [0]:
dic_type={}
dic_type[1]="str"
dic_type[2]="str"
for i in range(3,11691):
    dic_type[2]="float64"
ft2=pd.read_table("GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct", sep="\t", header=2,dtype=dic_type)
ft1=ft2.transpose()
ft1_new = ft1.drop("Description", axis=0).drop("Name", axis=0)
df2=pd.read_csv("answer.csv", delimiter="\t")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [30]:
merge=pd.merge(df2, ft1,left_on="SAMPID",right_index=True)

data/  seq_data.zip


In [0]:
df3=merge.iloc[:,:2]

In [0]:
ft3=merge[:].drop("SMTS", axis=1)

In [0]:
label = LabelEncoder()
#ラベルを覚えさせる
label = label.fit(df3['SMTS'])
#ラベルを整数に変換
df3['tissue'] = label.transform(df3['SMTS'])
df3.head()


In [0]:
X_train, X_test, y_train, y_test =train_test_split(ft3.iloc[:,1:],df3.iloc[:,2])
print(ft3.shape,X_train.shape,y_train.shape)
print(df3.shape,X_test.shape,y_test.shape)